### Data Import & Setup

In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [10]:
df = pd.read_csv(r"C:\Users\jkmfi\Documents\Quant_nifty_project\data\clean\nifty_features_5min.csv")

In [11]:
df = df.reset_index(drop=True)

In [12]:
print(df.shape)

(250, 21)


### Data Cleaning & Validation

In [13]:
# Safety check
df = df.dropna(subset=['EMA_5', 'EMA_15', 'spot_return'])

In [14]:
df.shape

(249, 21)

### Signal Generation (EMA Crossover)

In [15]:
df['EMA_Signal'] = 0

df.loc[
    (df['EMA_5'] > df['EMA_15']) &
    (df['EMA_5'].shift(1) <= df['EMA_15'].shift(1)),
    'EMA_Signal'
] = 1

df.loc[
    (df['EMA_5'] < df['EMA_15']) &
    (df['EMA_5'].shift(1) >= df['EMA_15'].shift(1)),
    'EMA_Signal'
] = -1

In [16]:
df = df.reset_index(drop=True)

### Position Management

In [17]:
df['Position'] = df['EMA_Signal'].replace(0, np.nan).ffill().fillna(0)

### Return & Strategy Calculation

In [18]:
df = df.reset_index(drop=True)

df['Return'] = np.log(df['Close_spot'] / df['Close_spot'].shift(1))

In [19]:
df['Strategy_Return'] = df['Position'].shift(1) * df['Return']

In [20]:
df[['Return', 'Strategy_Return']] = df[['Return', 'Strategy_Return']].fillna(0)

In [21]:
df[['Close_spot', 'EMA_5', 'EMA_15', 'EMA_Signal', 'Position', 'Return', 'Strategy_Return']].tail(10)

,Close_spot,EMA_5,EMA_15,EMA_Signal,Position,Return,Strategy_Return
239,26146.55,26076.641855,26030.442791,0,1.0,0.000648,0.000648
240,26328.55,26160.611237,26067.706192,0,1.0,0.006937,0.006937
241,26250.30,26190.507491,26090.530418,0,1.0,-0.002976,-0.002976
242,26178.70,26186.571661,26101.551616,0,1.0,-0.002731,-0.002731
243,26140.75,26171.297774,26106.451414,0,1.0,-0.001451,-0.001451
244,25876.85,26073.148516,26077.751237,-1,-1.0,-0.010147,-0.010147
245,25683.30,25943.199011,26028.444832,0,-1.0,-0.007508,0.007508
246,25790.25,25892.216007,25998.670478,0,-1.0,0.004156,-0.004156
247,25732.30,25838.910671,25965.374169,0,-1.0,-0.002250,0.002250
248,25665.60,25781.140448,25927.902398,0,-1.0,-0.002595,0.002595


### Train–Test Split

In [22]:
split = int(0.7 * len(df))

train = df.iloc[:split]
test  = df.iloc[split:]

### Equity Curve Construction

In [23]:
df['Equity'] = np.exp(df['Strategy_Return'].cumsum())

### Performance Metrics

In [24]:
total_return = df['Equity'].iloc[-1] - 1
total_return

-0.14463198069598404

In [25]:
total_trades = (df['Position'].diff() != 0).sum()
total_trades

22

In [26]:
sharpe_ratio = (
    np.sqrt(252) *
    df['Strategy_Return'].mean() /
    df['Strategy_Return'].std()
)

In [27]:
downside_returns = df[df['Strategy_Return'] < 0]['Strategy_Return']

sortino_ratio = (
    np.sqrt(252) *
    df['Strategy_Return'].mean() /
    downside_returns.std()
)

In [28]:
rolling_max = df['Equity'].cummax()
drawdown = (df['Equity'] - rolling_max) / rolling_max
max_drawdown = drawdown.min()

In [29]:
calmar_ratio = total_return / abs(max_drawdown)

In [30]:
winning_trades = (df['Strategy_Return'] > 0).sum()
losing_trades  = (df['Strategy_Return'] < 0).sum()

win_rate = winning_trades / (winning_trades + losing_trades)

In [31]:
gross_profit = df[df['Strategy_Return'] > 0]['Strategy_Return'].sum()
gross_loss   = abs(df[df['Strategy_Return'] < 0]['Strategy_Return'].sum())

profit_factor = gross_profit / gross_loss

### Trade Statistics

In [32]:
trade_durations = []
current_duration = 0

for i in range(1, len(df)):
    if df.loc[i, 'Position'] != 0:
        current_duration += 1
    else:
        if current_duration > 0:
            trade_durations.append(current_duration)
            current_duration = 0

average_trade_duration = (
    np.mean(trade_durations) if trade_durations else 0
)

In [36]:
metrics = {
    'Total Return (%)': total_return * 100,
    'Sharpe Ratio': sharpe_ratio,
    'Sortino Ratio': sortino_ratio,
    'Calmar Ratio': calmar_ratio,
    'Max Drawdown (%)': max_drawdown * 100,
    'Win Rate (%)': win_rate * 100,
    'Profit Factor': profit_factor,
    'Average Trade Duration (bars)': average_trade_duration,
    'Total Trades': total_trades
}

### Results Summary

In [37]:
pd.Series(metrics)

Total Return (%)                -14.463198
Sharpe Ratio                     -1.396653
Sortino Ratio                    -2.099750
Calmar Ratio                     -0.910675
Max Drawdown (%)                -15.881847
Win Rate (%)                     46.721311
Profit Factor                     0.784165
Average Trade Duration (bars)     0.000000
Total Trades                     22.000000
dtype: float64

In [34]:
print("Trades:", total_trades)
print("Non-zero returns:", (df['Strategy_Return'] != 0).sum())

Trades: 22
Non-zero returns: 244
